## DCF_1 ###

- Basic version of DCF 
- Acquire data from Yahoo finance using "yfinance"


step :
- get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash
- input > growth rate , terminal growth rate , discount rate , horizons
- calculate > FCF 
- calculate > terminal value
- calculate > present value
- calculate > Intrinsic value
- calculate > Margin of safety
- summary

In [50]:
import pandas as pd
import yfinance as yf

In [51]:
ticker = 'cpall.bk'
data = yf.Ticker(ticker)
data.info

{'address1': '58/28 THE TARA Building',
 'address2': 'Moo 2 Chaengwattana Road Bang Talad Pakkred',
 'city': 'Nonthaburi',
 'zip': '10500',
 'country': 'Thailand',
 'phone': '66 2 071 9000',
 'fax': '66 2 238 1767',
 'website': 'https://www.cpall.co.th',
 'industry': 'Grocery Stores',
 'industryKey': 'grocery-stores',
 'industryDisp': 'Grocery Stores',
 'sector': 'Consumer Defensive',
 'sectorKey': 'consumer-defensive',
 'sectorDisp': 'Consumer Defensive',
 'longBusinessSummary': 'CP ALL Public Company Limited, together with its subsidiaries, operates and franchises convenience stores under the 7-Eleven name to other retailers primarily in Thailand. It operates through four segments: Convenience stores, Wholesale, Retail and Mall, and Other. The company also operates frozen food plants and bakeries; distributes various commercial cards and tickets, catalog merchandises, and food and non-food products; and retails health and beauty products. In addition, it offers bill payment, software

In [52]:
### - get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash ###

## Shares ## 
shares = data.info['sharesOutstanding']
print('Ticker = ',ticker,' Shares = ',shares)

## Net Debt = Long Term Borrowings - Cash ###
totalnoncurrentliabilities = data.balancesheet.loc[data.balancesheet.index == 'Total Non Current Liabilities Net Minority Interest'].values[0][0] - data.balancesheet.loc[data.balancesheet.index == 'Cash And Cash Equivalents'].values[0][0]
print('Ticker = ',ticker,' Net Debt = ',totalnoncurrentliabilities)

## Average Free Cash Flow ## 
averagefreecashflow = data.cash_flow.T['Free Cash Flow'].dropna().mean()
print('Ticker = ',ticker,' Free Cash Flow = ',averagefreecashflow)

Ticker =  cpall.bk  Shares =  8983000064
Ticker =  cpall.bk  Net Debt =  319465177306.0
Ticker =  cpall.bk  Free Cash Flow =  37208261575.0


In [53]:
## - input > growth rate , terminal growth rate , discount rate , horizons ##

year_growth = 5
year_sustain = 5

discountrate = 0.1
terminalgrowthrate = 0.03

FCF = data.cash_flow.T['Operating Cash Flow'].dropna()
FCF = pd.DataFrame(FCF)
FCF = FCF.sort_index(ascending=True)
FCF['diff'] = FCF['Operating Cash Flow'].diff(1)
FCF['growth'] = FCF['diff']/FCF['Operating Cash Flow']

growthrate = FCF['growth'].mean()
print(growthrate)

0.23011723669834674


In [58]:
## - calculate > FCF ##

FCF_year_growth = [averagefreecashflow * ((1+growthrate)**i) for i in range(1,year_growth+1,1)]
print(FCF_year_growth)

FCF_year_sustain = [FCF_year_growth[-1] * ((1+(growthrate/2))**i) for i in range(1,year_sustain+1,1)]
print(FCF_year_sustain)

[np.float64(45770523910.98828), np.float64(56303110395.62051), np.float64(69259426577.38268), np.float64(85197214436.68202), np.float64(104802561997.24777)]
[np.float64(116860999980.10468), np.float64(130306865176.71829), np.float64(145299793045.3635), np.float64(162017786479.58386), np.float64(180659329139.91614)]
